<a href="https://colab.research.google.com/github/antonellagambarte/procesamiento_del_lenguaje_I/blob/main/Desafio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 30.4 MB/s eta 0:00:00


In [2]:
# Descarga de carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-11-08 00:19:34--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-08 00:19:34--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-08 00:19:34--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
os.listdir("./songs_dataset/")

['beatles.txt',
 'amy-winehouse.txt',
 'lady-gaga.txt',
 'lorde.txt',
 'bruce-springsteen.txt',
 'blink-182.txt',
 'dickinson.txt',
 'al-green.txt',
 'bjork.txt',
 'cake.txt',
 'bruno-mars.txt',
 'nirvana.txt',
 'bieber.txt',
 'lil-wayne.txt',
 'rihanna.txt',
 'lin-manuel-miranda.txt',
 'missy-elliott.txt',
 'janisjoplin.txt',
 'radiohead.txt',
 'britney-spears.txt',
 'leonard-cohen.txt',
 'kanye.txt',
 'kanye-west.txt',
 'Lil_Wayne.txt',
 'notorious-big.txt',
 'bob-marley.txt',
 'jimi-hendrix.txt',
 'johnny-cash.txt',
 'bob-dylan.txt',
 'ludacris.txt',
 'Kanye_West.txt',
 'adele.txt',
 'patti-smith.txt',
 'prince.txt',
 'nickelback.txt',
 'dj-khaled.txt',
 'r-kelly.txt',
 'dr-seuss.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'michael-jackson.txt',
 'nicki-minaj.txt',
 'drake.txt',
 'disney.txt',
 'notorious_big.txt',
 'paul-simon.txt',
 'nursery_rhymes.txt',
 'eminem.txt',
 'dolly-parton.txt']

In [4]:
df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-2822025462.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)


,0
0,[Spoken Intro:]
1,You ever want something
2,that you know you shouldn't have
3,"The more you know you shouldn't have it,"
4,The more you want it


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby(0).size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9484
